In [ ]:
import json
import numpy as np
import pandas as pd
with open('./py150/python50k_eval.json') as data_file:
    all_data = [json.loads(r) for r in data_file.readlines()]

In [ ]:
len(all_data)

In [7]:
def find_item(data):
    
    functions = [(i,text) for i, text in enumerate(data) if text['type'] == 'FunctionDef']
    
    full_list = {}
    for (index, f) in functions:
        ans = []
        row = f['children']
        while row:
            ans.append(row)
            row = [node for root in row for node in data[root].get('children',[]) if node]
        full_list[index] =  (f,ans)
    
    return full_list

def find_expr_str(List, data):
    
    ans = []
    for cand_id in range(len(List)):
        for i in List[cand_id]:
            if data[i]['type'] == 'Expr':
                if data[data[i]['children'][0]]['type'] == 'Str':
                    ans.append((i, data[i]['children'][0], data[data[i]['children'][0]]))
    return ans

In [10]:
def find_def_docstring(data):
    ans = []
    full_list = find_item(data)
    
    for k, v in full_list.items():
        if find_expr_str(v[1], data):
            docstring = find_expr_str(v[1], data)
            ans.append((full_list[k][0]['value'], docstring[0][2]['value']))
    return ans

In [11]:
function_info = []
for i, value in enumerate(all_data):
    func_doc = find_def_docstring(value)
    if func_doc:
        for tuples in func_doc:
            function_info.append([i, tuples[0], tuples[1]])

In [12]:
df = pd.DataFrame(np.array(function_info), columns=['data_id', 'function_name', 'docstring'])

In [13]:
df.shape

(264, 3)

In [14]:
df.head()

,data_id,function_name,docstring
0,8,ping,Handle ping requests
1,8,message,Proxy message from one user to another
2,8,presence,Presence information may be sent out from the ...
3,8,roster,A roster is this account's list of contacts; i...
4,8,push,Push roster changes to all clients that have r...


In [15]:
df.to_pickle("./dummy.pkl")

In [ ]:
### unpickled_df = pd.read_pickle("./dummy.pkl")
### unpickled_df